# Catalogue download _(Avoid Row limit)_

__DESCRIPTION__

Script to describe different ways to obtain by TAP a big subset of data.

@authors Javier Hernandez & Tamara Civera(CEFCA)

## Download by __Tile__

Use when you want to download a __big subset__ of the catalogue database.

Fist, obtain __tile ids__ for __dual mode__:

In [ ]:
import pyvo as vo

service = vo.dal.TAPService("https://archive.cefca.es/catalogues/vo/tap/jplus-dr2")
resultset = service.search("SELECT tile_id FROM TileImage WHERE filter_id = jplus::rSDSS")
ref_tiles = resultset["tile_id"]
print("Total reference tiles", len(ref_tiles))

Download in __CSV__ format with __simplevotap__ module

In [ ]:
from simplevotap import ADQLService

resultset = service.search("SELECT tile_id FROM TileImage WHERE filter_id = jplus::rSDSS AND FWHMG < .88")
ref_tiles = resultset["tile_id"]
print("Total reference tiles:", len(ref_tiles))

ser = ADQLService("http://archive.cefca.es/catalogues/vo/tap/jplus-dr2")
sql_base = "SELECT tile_id, number, ALPHA_J2000, DELTA_J2000, CLASS_STAR, MAG_AUTO, MAG_ERR_AUTO FROM MagABDualObj"
for t in ref_tiles:
    adql = sql_base + " WHERE tile_id = " + str(t) + " AND ARRAY_MAX_INT(flags) = 0 ORDER BY number"
    ser.exec_async(adql, filename='{0}.csv'.format(t), formattable='csv', delete=True)
    print("Done", t)

print("All done.")


Another option: download in __FITs__ format by __tile blocks__ (more compact and with less files)

In [ ]:
resultset = service.search("SELECT tile_id FROM TileImage WHERE filter_id = jplus::rSDSS AND FWHMG < .9")
ref_tiles = resultset["tile_id"]

blocks = int(len(ref_tiles) / 10)
if len(ref_tiles) % 10 > 0:
    blocks += 1
print("Total reference tiles:", len(ref_tiles), "Blocks:", blocks)
    
sql_base = "SELECT tile_id, number, ALPHA_J2000, DELTA_J2000, CLASS_STAR, MAG_AUTO, MAG_ERR_AUTO FROM MagABDualObj"
for i in range(blocks):
    grp = list(ref_tiles[i * 10:(i+1) * 10])
    adql = sql_base + " WHERE tile_id IN (" + ','.join([str(x) for x in grp]) + ") AND ARRAY_MAX_INT(flags) = 0"
    ser.exec_async(adql, filename='block{0}.fits'.format(i), formattable='fits', delete=True)
    print("Done block", i)

print("All done.")

## Download with LIMIT and OFFSET

Use it when you want a __small subset__ but it surpass the TAP row limit.

In [ ]:
service = vo.dal.TAPService("https://archive.cefca.es/catalogues/vo/tap/jplus-dr2")

# 69209 objects between dec -1 and 0
sql = """SELECT TOP {0} tile_id, number, ALPHA_J2000, DELTA_J2000, CLASS_STAR, MAG_AUTO, MAG_ERR_AUTO 
FROM MagABDualObj 
WHERE DELTA_J2000 BETWEEN (-1) AND 0 ORDER BY tile_id, number OFFSET {1}"""

top = 50000
offset = 0
while True:
    adql = sql.format(top, offset)
    print(adql)
    resultset = service.run_async(adql, maxrec=1000000)
    found = len(resultset)
    print("found", found, "rows")
    # Add to result file ...
    if found < top:
        break
    offset += top

print("Done!")